In [ ]:
import torch
from safetensors.torch import load_file

LOAD_PATH = "/Users/blacksamorez/models/Llama-2-7b-AQLM-PV-2Bit-2x8-hf/model.safetensors"
SAVE_PATH = "/Users/blacksamorez/models/Llama-2-7b-AQLM-PV-2Bit-2x8-hf/executorch.pth"

dict = load_file(LOAD_PATH)

mapping = {
    "model.": "",
    
    "self_attn.q_proj": "attention.wq",
    "self_attn.k_proj": "attention.wk",
    "self_attn.v_proj": "attention.wv",
    "self_attn.o_proj": "attention.wo",
    
    "mlp.up_proj": "feed_forward.w3",
    "mlp.gate_proj": "feed_forward.w1",
    "mlp.down_proj": "feed_forward.w2",
    
    "input_layernorm": "attention_norm",
    "post_attention_layernorm": "ffn_norm",
    
    "lm_head": "output",
    "embed_tokens": "tok_embeddings",
}


new_dict = {}

for key, value in dict.items():
    for old, new in mapping.items():
        key = key.replace(old, new)
        
    if "attention.wq.codes" in key or "attention.wk.codes" in key:
        # [num_out_groups, num_in_groups, num_codebooks]
        print(f"Transposing codes {key} {value.shape=}")
        value = (value.reshape(32, 2, 128 // 2, -1, 2)
            .transpose(1, 2)
            .reshape(128 * 32, -1, 2))
        
    if "attention.wq.scales" in key or "attention.wk.scales" in key:
        # [num_out_groups, 1, 1, 1]
        print(f"Transposing scales {key} {value.shape=}")
        value = (value.reshape(32, 2, 128 // 2, 1)
            .transpose(1, 2)
            .reshape(128 * 32, 1, 1, 1))
    
    new_dict[key] = value
    
del new_dict["output.weight"]
del new_dict["tok_embeddings.weight"]

torch.save(new_dict, SAVE_PATH)